# Notebook to compute the Degree Lighting Week 'DLW' index 

In [ ]:
# load libraries
import pandas as pd
import numpy as np

In [ ]:
# For SeaWiFis and modis (1998-2017)
# Import file with processed data in .h5 format and open it as a dataframe 'df'
## NOTE: This data can be avoided if a dtaframe is already created ##
import pandas as pd
with pd.HDFStore('df_PAR.h5','r') as input:
    df_PAR=input.get('df_PAR')

In [ ]:
df_PAR.head()

In [ ]:
def arrange_df (df):
    'This function reindex, sort_values, interpolate NaN´s and append the average of the week and average of month'
    df = df.reindex(columns=['site_name','year','month','week','days','par'])
    df = df.sort_values(by=['site_name','year','month','week','days'])
    df=df.interpolate()
    df['par_week'] = df.groupby(['site_name','year','month','week'])['par'].transform('mean')
    df['par_month'] = df.groupby(['site_name','year','month'])['par'].transform('mean') 
    df['90th_percentile'] = df.groupby(['site_name','year','month'])['par'].transform(lambda x: x.drop_duplicates().quantile(0.9))

    ## Compute diff metrics of anomalies for sensitivity analysis

    #df['par_Mclim'] = df.groupby(['site_name','month'])['par'].transform('mean')
    df['par_Wclim'] = df.groupby(['site_name','week'])['par'].transform('mean')
    df['M_anomaly'] = df['par_month'] - df['par_Mclim']  
    df['90_anomaly'] = df['par_week'] - df['90th_percentile'] 
    df['W_anomaly'] = df['par_week'] - df['par_Wclim']  
    return df

In [ ]:
df_PAR = arrange_df(df_PAR)

# Degree Lighting Week 'DLW'

In [ ]:
# To compute DLW use one anomaly metric; in this case W_anomaly 
W_anomaly = df_PAR.groupby(['site_name','year','week'])['par_week'].mean()-df_PAR.groupby(['site_name','year','week'])['par_Wclim'].mean()

In [ ]:
W_anomaly = round(W_anomaly, 3)
hit = W_anomaly < 0
W_anomaly[hit] = 0

In [ ]:
W_anomaly = pd.DataFrame(W_anomaly)

In [ ]:
def DLW (df):
     ' function to compute the DLW index; it sum only values < anomaly in a rolling window'
    df = round(df.rolling(center=False,window=12).sum(),3) # Sum the values in the windows of "X" weeks; it can be edited to test different "windows"
    df['site_name'] = df.index.get_level_values(level=0) # assign names and add new columns based on hiden levels
    df['year'] = df.index.get_level_values(level=1)
    df['week'] = df.index.get_level_values(level=2)
    df.columns = ['DLW','site_name','year','week'] # Rename columns
    df = df.reset_index(level=(0,1,2), drop=True) # Drop the hiden levels
    df = df.reindex(columns = ['site_name','year','week','DLW']) # Re-order the columns 
    return df

In [ ]:
DLW = DLW(W_anomaly)

In [ ]:
DLW.to_csv('DLW.csv')